In [1]:
import numpy as np
import tensorflow as tf

In [7]:
m1 = np.array([[1, 1.4, 0], [2, 3, 3]])
m2 = np.array([[2, 3], [1, 5]])

In [9]:
tm1 = tf.placeholder(shape=m1.shape, dtype=tf.float32)
tm2 = tf.placeholder(shape=m2.shape, dtype=tf.float32)

In [11]:
def tf_kronecker_product(tm1, tm2):
    kprod0 = tf.multiply(tm1[0, 0], tm2)

    j0 = tf.constant(1)
    _, kprod = tf.while_loop(lambda j, m: tf.less(j, tm1.shape[1]),
                             lambda j, m: [j+1, tf.concat([m, tf.multiply(tm1[0, j], tm2)], axis=1)],
                             loop_vars=[j0, kprod0],
                             shape_invariants=[j0.get_shape(), tf.TensorShape([tm1.shape[0], None])])
    
    for i in range(1, tm1.shape[0]):
        kprod0 = tf.multiply(tm1[i, 0], tm2)

        j0 = tf.constant(1)
        _, kprod1 = tf.while_loop(lambda j, m: tf.less(j, tm1.shape[1]),
                                  lambda j, m: [j+1, tf.concat([m, tf.multiply(tm1[i, j], tm2)], axis=1)],
                                  loop_vars=[j0, kprod0],
                                  shape_invariants=[j0.get_shape(), tf.TensorShape([tm1.shape[0], None])])

        kprod = tf.concat([kprod, kprod1], axis=0)
        
    return kprod

In [12]:
kprod = tf_kronecker_product(tm1, tm2)

In [13]:
sess = tf.Session()
init = tf.global_variables_initializer()

sess.run(init)
sess.run(kprod, feed_dict={tm1: m1, tm2: m2})

array([[ 2. ,  3. ,  2.8,  4.2,  0. ,  0. ],
       [ 1. ,  5. ,  1.4,  7. ,  0. ,  0. ],
       [ 4. ,  6. ,  6. ,  9. ,  6. ,  9. ],
       [ 2. , 10. ,  3. , 15. ,  3. , 15. ]], dtype=float32)